In [1]:
import sys
import os

# Data access
import pandas as pd
import json
# from google.colab import files
from IPython.display import display, HTML

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
# import missingno as msno

import numpy as np
# Data manipulation 
import re
import math

In [12]:

def model_sentences_preprocessing(sentences, nlp_gender_counts_csv="./data/male_female_counts_ok.csv", output_folder="../pipeline/data/"):
    """
    Preprocessing the sentences and include male, female counts from the NLP model.
    Returns a pandas DataFrame and stores a CSV 
    """
    sentences_df = pd.read_csv(sentences)
    sentences_df = sentences_df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
    sentences_df.replace('None',np.NaN, inplace=True)

    sentences_df_subset = sentences_df[sentences_df['tokenized_METHODS'].notna()]
    sentences_df.set_index("PMCID", inplace=True)

    try: 
        counts_df = pd.read_csv(nlp_gender_counts_csv)
        final_df_small = pd.merge(sentences_df, counts_df, on=["PMCID"])
        final_df = pd.merge(sentences_df, counts_df, how="outer", on=["PMCID"])

        final_df.set_index("PMCID", inplace=True)
        final_df_small.set_index("PMCID", inplace=True)

    except FileNotFoundError:
        final_df = sentences_df.copy()
        final_df_small = sentences_df.copy()


    final_df['subject'] = final_df['subject'].apply(lambda x: np.nan if x == "[]" else x )
    final_df['subject'] = final_df['subject'].apply(lambda x: x.replace("[","").replace("]","").replace("'", "").replace("\\n", "").replace("\\n ", "") if x != "[]" and not isinstance(x, float) else x )

    final_df_small['subject'] = final_df_small['subject'].apply(lambda x: np.nan if x == "[]" else x )
    final_df_small['subject'] = final_df_small['subject'].apply(lambda x: x.replace("[","").replace("]","").replace("'", "").replace("\\n", "").replace("\\n ", "") if x != "[]" and not isinstance(x, float) else x )


    # some rows repeat, as they contain different sentences from a single PMCID in the "tokenized_METHODS" column -- but they contain the same "count" from the model
    # drop it and delete identical rows
    final_df.drop(["tokenized_METHODS", "Unnamed: 0"], axis=1, inplace=True)
    final_df.drop_duplicates(inplace=True)

    final_df_small.drop(["tokenized_METHODS", "Unnamed: 0"], axis=1, inplace=True)
    final_df_small.drop_duplicates(inplace=True)


    # print(sentences_df.shape, sentences_df_subset.shape, counts_df.shape, final_df.shape, final_df_small.shape)
    final_df.to_csv(f"{output_folder}final_dataframe.csv", sep=',', encoding='utf-8', index=False)
    final_df_small.to_csv(f"{output_folder}final_dataframe_subset.csv", sep=',', encoding='utf-8', index=False)


    # display(HTML(final_df.head(10).to_html()))
    
    return final_df


final_sentences_df = model_sentences_preprocessing(sentences="../pipeline/data/candidate_dataframe_extended_test.csv")

## adding gender of authors


In [17]:
print(final_sentences_df.columns)

import gender_guesser.detector as gender



d = gender.Detector()
print(d.get_gender("Bob"))



# def add_gender_info(input_csv, output_folder):
#     """
#     """

    




Index(['last_author', 'subject', 'keywords', 'journal-title', 'publisher-name',
       'article-title', 'first_author', 'contrib', 'aff', 'year'],
      dtype='object')


ModuleNotFoundError: No module named 'gender_guesser'